In [1]:
import os
from pathlib import Path
os.chdir(os.path.abspath("./.."))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
import datetime

from utils.data_io import *
from utils.common import *

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Masking, Dropout, Dense, Conv1D, GRU, LSTM, BatchNormalization
import tensorboard
import tensorflow_addons as tfa

tf.config.run_functions_eagerly(True)
# tf.config.threading.set_inter_op_parallelism_threads(0)

from src.dx5849_training_data_3src import *


def data_zero_masking(data):
    data[np.isnan(data)] = 0
    return data

root_folder = os.path.abspath('.')
data_folder = os.path.join(root_folder, 'processed/patient_data/dx_pred_0_12_3src')

/home/dhm/workspace/conference_projects/MICCAI2022_AKF_feature_importance


In [7]:
def generate_balanced_dataset_raw(sample_list, sample_label,
                                  file_folder=os.path.join(os.path.abspath('.'), 'processed/patient_data/dx_pred_0_12_3src/ts_aligned'),
                                  resample_freq=None):
    
    path = {
        'pos': os.path.join(file_folder, 'pos/data'),
        'neg': os.path.join(file_folder, 'neg/data'),
    }
    info_path = {
        'pos': os.path.join(file_folder, 'pos/info')
        'neg': os.path.join(file_folder, 'neg/info'),
    }
    
    N_sample = {}
    N_sample['total'] = len(sample_label)
    N_sample['pos'] = sum(sample_label)
    N_sample['neg'] = N_sample - N_sample_pos

    idx = {
        'pos': np.where(np.array(sample_label) == 1)[0],
        'neg': np.where(np.array(sample_label) == 0)[0]
    } 
    
    info_set = {'pos': [], 'neg':[]}
    data_set = {'pos': [], 'neg':[]}
    label_set = {'pos': [], 'neg':[]}         
    
    tf_dataset = {
        'pos': {},
        'neg': {},
    }
    
    for data_label in ['pos', 'neg']:
        for i in tqdm(range(N_sample[data_label])):
            info = pd.read_csv(os.path.join(info_path[data_label], sample + '.csv'), sep=';', header=0, index_col=0)
            data = pd.read_csv(os.path.join(path[data_label], sample + '.csv'), sep=';', header=0, index_col=0)
            
            if resample_freq:
                data = resample_aligned_data(data, resample_freq)

            info = info.to_numpy().flatten()
            data = data.to_numpy()
            info_set[data_label].append(info)
            data_set[data_label].append(data)
            label_set[data_label].append(label)
            
        tf_dataset[data_label] = tf.data.Dataset.from_tensor_slices(({'static_inputs': info_set[data_label],
                                                                     'time_series_inputs': data_set[data_label]}, 
                                                                     train_labels))             
            

    return tf_dataset

b'/home/dhm/workspace/conference_projects/MICCAI2022_AKF_feature_importance/processed/patient_data/dx_pred_0_12_3src/raw/neg/info/1679493_0.csv'
b'/home/dhm/workspace/conference_projects/MICCAI2022_AKF_feature_importance/processed/patient_data/dx_pred_0_12_3src/raw/neg/info/215476_0.csv'
b'/home/dhm/workspace/conference_projects/MICCAI2022_AKF_feature_importance/processed/patient_data/dx_pred_0_12_3src/ts_aligned/pos/data/1029907_2.csv'
b'/home/dhm/workspace/conference_projects/MICCAI2022_AKF_feature_importance/processed/patient_data/dx_pred_0_12_3src/ts_aligned/pos/data/790396_0.csv'
b'/home/dhm/workspace/conference_projects/MICCAI2022_AKF_feature_importance/processed/patient_data/dx_pred_0_12_3src/raw/neg/info/1291125_0.csv'
b'/home/dhm/workspace/conference_projects/MICCAI2022_AKF_feature_importance/processed/patient_data/dx_pred_0_12_3src/raw/neg/data/3155303_0.csv'
b'/home/dhm/workspace/conference_projects/MICCAI2022_AKF_feature_importance/processed/patient_data/dx_pred_0_12_3src/r